# Create VIEWs on SEC DERA data by LEI

## Full-year FY REVENUES, INCOME

## Point-in-time FLOAT, DEBT, CASH, ASSETS, (EV = FLOAT + DEBT - CASH, EVIC = FLOAT + DEBT)

In this workflow we assume (and depend upon) the interesting NUM data coming only from the consolidated entity (NUM.COREG IS NULL) and not from co-registrants.  
Please explore EXCELON (EXC) and DOMINION ENERGY (D) to see examples of reports with substantial co-registrant entities.

In [ ]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Set session variable CATALOG to make query terms much more compact

In [ ]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

import pandas as pd

In [ ]:
from osc_ingest_trino import *

trino_bucket = attach_s3_bucket("S3_DEV")

In [ ]:
for tbl in ['t_r', 't_f', 't_d', 't_c', 't_a', 't_i']:
    qres = engine.execute(f"drop table if exists sec_dera.{tbl}")
    print(qres.fetchall())

### FX

Many public companies are non-US companies or report facts in units of measurement other than USD.  We build a small table of market knowledge so that we can do foreign exchange (FX) calculations.

In [ ]:
sql = ("""
select count (*) as count, uom, ddate, max(value) as maxval, min(value) as minval from sec_dera.num
where ddate>=DATE('2019-01-01')
      and (tag='ClosingForeignExchangeRate' or tag='ForeignCurrencyExchangeRateTranslation1')
      and qtrs=0 and uom!= 'USD' and (uom like '%USD%' or length(uom)=3)
group by uom, ddate, tag
order by ddate, uom, tag
""")
df_closing_fx = pd.read_sql (sql, engine, parse_dates=['ddate'])
# display(df_closing_fx)

sql = ("""
select count (*) as count, uom, ddate, max(value) as maxval, min(value) as minval from sec_dera.num
where ddate>=DATE('2019-01-01')
      and tag='AverageForeignExchangeRate'
      and qtrs=4 and uom!= 'USD' and (uom like '%USD%' or length(uom)=3)
group by uom, ddate, tag
order by ddate, uom, tag
""")
df_average_fx = pd.read_sql (sql, engine, parse_dates=['ddate'])
# display(df_average_fx)

In [ ]:
import re

def create_fx_dict(df):
    """From a list of FX pairs or non-USD (presumed to convert to USD) FX rates by dates, return a dictionary of FX rates by dates"""
    df_gbp = df[df.uom.str.contains(r'GBP', flags=re.I)]
    df_eur = df[df.uom.str.contains(r'EUR', flags=re.I)]
    df_other = df[~df.uom.str.contains(r'(GBP)|(EUR)', flags=re.I)]
    
    fx_input = {'GBP': df_gbp, 'EUR': df_eur }

    fx_df = {}

    for fx in ['GBP', 'EUR']:
        df_base = fx_input[fx].loc[df.maxval>1, ['ddate', 'maxval']].groupby(by='ddate').mean()
        df_base['minval'] = 1/df_base.maxval
        df_quote = fx_input[fx].loc[df.minval<1, ['ddate', 'minval']].groupby(by='ddate').mean()
        df_quote['maxval'] = 1/df_quote.minval
        df_fx = pd.concat((df_base, df_quote), axis=0).groupby(by='ddate').mean().reset_index()
        df_fx['base'] = fx
        df_fx.rename(columns={'maxval':'to_USD', 'minval':'from_USD'}, inplace=True)
        df_fx = df_fx[['base', 'to_USD', 'from_USD', 'ddate']]
        fx_df[fx] = df_fx.convert_dtypes()
    
    for fx in list({x[int(x[-1]=='USD')-1].upper():x[int(x[0]=='USD')-1].upper() for x in df_other.uom.str.split(r'[-_/]').values }.keys()):
        df_other_fx = df_other[df_other.uom.str.contains(fx, flags=re.I)]
        df_base = df_other_fx.loc[df.maxval>1, ['ddate', 'maxval']].groupby(by='ddate').mean()
        df_base['minval'] = 1/df_base.maxval
        df_quote = df_other_fx.loc[df.minval<1, ['ddate', 'minval']].groupby(by='ddate').mean()
        df_quote['maxval'] = 1/df_quote.minval
        df_fx = pd.concat((df_base, df_quote), axis=0).groupby(by='ddate').mean().reset_index()
        df_fx['base'] = fx
        df_fx.rename(columns={'maxval':'from_USD', 'minval':'to_USD'}, inplace=True)
        df_fx = df_fx[['base', 'to_USD', 'from_USD', 'ddate']]
        fx_df[fx] = df_fx.convert_dtypes()
    
    return fx_df

The `closing_fx` table contains point-in-time FX market information.

The `average_fx` table contains a four-quarter average of FX rates.

In [ ]:
# TODO: change this to use osc_trino_tools.ingest_unmanaged_parquet

import io

ingest_schema = 'sec_dera'

def create_fx_table(df_fx, tbl):
    qres = engine.execute(f"drop table if exists {ingest_schema}.{tbl}")
    display(qres.fetchall())

    df = enforce_sql_column_names(pd.concat(create_fx_dict(df_fx).values()))

    buf = io.BytesIO()
    df.to_parquet(path=buf)
    buf.seek(0)
    trino_bucket.upload_fileobj(Fileobj=buf,
                                Key=f'trino/{ingest_schema}/{tbl}/data.parquet')

    columnschema = create_table_schema_pairs(df, typemap={'datetime64[ns]':'timestamp(3)'})
    tabledef = f"""
create table if not exists {ingest_schema}.{tbl} (
{columnschema}
) with (
format = 'parquet',
external_location = 's3a://{trino_bucket.name}/trino/{ingest_schema}/{tbl}/'
)
"""
    print(tabledef)

    table_create = engine.execute(tabledef)
    # display(table_create.fetchall())

    if False:
        dataset_query = (f'SELECT * FROM {ingest_schema}.{tbl} limit 10')
        print(dataset_query)
        dataset = engine.execute(dataset_query)
        display(dataset.fetchall())

for df, tbl in [ (df_closing_fx,'closing_fx'), (df_average_fx,'average_fx') ]:
    create_fx_table(df, tbl)

### Annual Tables: Revenue and Income

We  only ingest annual data (NUM.QTRS=4) for Revenues and Income.  

Many companies report not only the present fiscal year data but two prior years as well.  
The row_number() code ensures we use the latest reported fact for a given year

In [ ]:
common_dera_form_test = "(S.form='10-K' or S.form='20-F' or S.form='40-F')"
common_dera_fy_test = "S.fy>=DATE('2018-01-01')"
common_dera_columns = 'S.adsh, S.cik, S.name, S.lei, S.sic, S.fy, N.ddate, N.uom'

In [ ]:
def generate_annual_dera_usd_query(what, tags):
    usd_query = f"""
create or replace view sec_dera.fy_{what}_usd_by_lei as
select {common_dera_columns}, max(value) as {what}
from sec_dera.sub as S join sec_dera.num as N on S.adsh=N.adsh
where {common_dera_form_test}
      and {common_dera_fy_test}
      and uom='USD'
      and coreg is NULL
      and qtrs=4
      and ({' or '.join(["N.tag='"+tag+"'" for tag in tags])})
group by {common_dera_columns}
"""
    return usd_query

def generate_annual_dera_xyz_query(what, tags):
    # The X term prevents us from collecting foreign currency info that's duplicative of USD info we prefer
    xyz_query = f"""
create or replace view sec_dera.fy_{what}_xyz_by_lei as
select {common_dera_columns}, max(N.value) as {what}
from sec_dera.sub as S left join sec_dera.fy_{what}_usd_by_lei X on S.adsh=X.adsh
     join sec_dera.num as N on S.adsh=N.adsh and (X.adsh IS NULL or X.ddate=N.ddate)
where X.ddate IS NULL
      and {common_dera_form_test}
      and {common_dera_fy_test}
      and N.uom!='USD'
      and coreg is NULL
      and qtrs=4
      and ({' or '.join(["N.tag='"+tag+"'" for tag in tags])})
group by {common_dera_columns}
"""
    return xyz_query

def generate_annual_dera_combined_query(tbl, what, month_list):
    query = f"""
create table sec_dera.{tbl} as
select {re.sub(r'[SN].','V_USD.',common_dera_columns)}, V_USD.{what} as {what}_usd
from (select *,
             row_number() over (partition by cik, ddate order by fy desc) as rn
      from sec_dera.fy_{what}_usd_by_lei) as V_USD
where rn=1
UNION ALL
select {re.sub(r'[SN].','V_XYZ.',common_dera_columns)},
       coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.{what} as {what}_usd
from (select *,
             row_number() over (partition by cik, ddate order by fy desc) as rn
      from sec_dera.fy_{what}_xyz_by_lei) as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     {' '.join(str(nmonths).join(["left join sec_dera.closing_fx FX_", "mo on V_XYZ.uom=FX_", "mo.base and date_diff('month', FX_", "mo.ddate, V_XYZ.ddate)=", ""]) for nmonths in month_list)}
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
where rn=1
"""
    return query

# print(generate_annual_dera_combined_query('t_r', 'revenue', [1, 2, 3]))

In [ ]:
def generate_instant_dera_usd_query(what, tags):
    usd_query = f"""
create or replace view sec_dera.{what}_usd_by_lei as
select {common_dera_columns}, max(value) as {what}
from sec_dera.sub as S
     join sec_dera.num as N on S.adsh=N.adsh
where {common_dera_form_test}
      and {common_dera_fy_test}
      and value>0
      and uom='USD'
      and coreg is NULL
      and qtrs=0
      and ({' or '.join(["N.tag='"+tag+"'" for tag in tags])})
group by {common_dera_columns}
"""
    return usd_query

def generate_instant_dera_xyz_query(what, tags):
    xyz_query = f"""
create or replace view sec_dera.{what}_xyz_by_lei as
select {common_dera_columns}, max(value) as {what}
from sec_dera.sub as S
     left join sec_dera.{what}_usd_by_lei X on S.adsh=X.adsh
     join sec_dera.num as N on S.adsh=N.adsh and (X.adsh IS NULL or X.ddate=N.ddate)
where X.ddate IS NULL
      and {common_dera_form_test}
      and {common_dera_fy_test}
      and value>0
      and N.uom!='USD'
      and coreg is NULL
      and qtrs=0
      and ({' or '.join(["N.tag='"+tag+"'" for tag in tags])})
group by {common_dera_columns}
"""
    return xyz_query

def generate_instant_dera_combined_query(tbl, what, month_list):
    query = f"""
create table sec_dera.{tbl} as
select {re.sub(r'[SN].','V_USD.',common_dera_columns)}, V_USD.{what} as {what}_usd
from (select *,
             row_number() over (partition by cik, ddate order by fy desc) as rn
      from sec_dera.{what}_usd_by_lei) as V_USD
where rn=1
UNION ALL
select {re.sub(r'[SN].','V_XYZ.',common_dera_columns)},
       coalesce(FX.to_usd, FX_1mo.to_usd, FX_2mo.to_usd, FX_3mo.to_usd, FX_avg.to_usd, FX_avg_py.to_usd, FX_avg_1231.to_usd) * V_XYZ.{what} as {what}_usd
from (select *,
             row_number() over (partition by cik, ddate order by fy desc) as rn
      from sec_dera.{what}_xyz_by_lei) as V_XYZ
     left join sec_dera.closing_fx FX on V_XYZ.uom=FX.base and V_XYZ.ddate=FX.ddate
     {' '.join(str(nmonths).join(["left join sec_dera.closing_fx FX_", "mo on V_XYZ.uom=FX_", "mo.base and date_diff('month', FX_", "mo.ddate, V_XYZ.ddate)=", ""]) for nmonths in month_list)}
     left join sec_dera.average_fx FX_avg on V_XYZ.uom=FX_avg.base and V_XYZ.ddate=FX_avg.ddate
     left join sec_dera.average_fx FX_avg_py on V_XYZ.uom=FX_avg_py.base and V_XYZ.ddate=date_add('year', 1, FX_avg_py.ddate)
     left join sec_dera.average_fx FX_avg_1231 on V_XYZ.uom=FX_avg_1231.base and DATE(cast (year(V_XYZ.ddate)-1 as varchar)||'-12-31')=FX_avg_1231.ddate
where rn=1
"""
    return query

print(generate_instant_dera_combined_query('t_f', 'float', [1, 2, 3]))

In [ ]:
revenue_tags = [
    'Revenue','Revenues','RevenueFromContractsWithCustomers',
    'RevenueFromContractWithCustomerIncludingAssessedTax',
    'RevenueFromContractWithCustomerExcludingAssessedTax',
    'RevenuesNetOfInterestExpense',
    'RegulatedAndUnregulatedOperatingRevenue',
    'RegulatedOperatingRevenuePipelines'
]

income_tags = [
    'ProfitLoss','NetIncomeLoss','ComprehensiveIncome'
]

dera_annual_dict = {
    'revenue': revenue_tags,
    'income': income_tags,
}


for what, tags in dera_annual_dict.items():
    tbl = 't_' + what[0]
    print(f'Creating annual table `{tbl}` for `{what}`')
    
    query_usd = generate_annual_dera_usd_query(what, tags)
    # print(query_usd)
    
    query_xyz = generate_annual_dera_xyz_query(what, tags)
    # print(query_xyz)
    
    qres = engine.execute(f"drop table if exists sec_dera.{tbl}")
    # display(qres.fetchall())
    
    qres = engine.execute(query_usd)
    # display(qres.fetchall())

    qres = engine.execute(query_xyz)
    # display(qres.fetchall())

    query_final = generate_annual_dera_combined_query(tbl, what, [1, 2, 3])
    qres = engine.execute(query_final)
    # print(query_fintal)
    display(qres.fetchall())

### Instant Tables: Assets, Cash, Debt, and Float (market cap)

These tables all measure point-in-time values.  We only measure the points-in-time that are part of annual reports. 

Many companies report not only the present fiscal year data but two prior years as well.  
The row_number() code ensures we use the latest reported fact for a given year

TODO: The float calculation reads the reported overall float without regard to share class.  For companies that have multiple associated tickers, this gives wrong results.  And probably very wrong results for BRK.A vs. BRK.B
For starters, look at NG (National Grid plc), NGG (US Depository of NG), and its various preferred classes

In [ ]:
float_tags = [
    'EntityPublicFloat',
    'FreeFloat',
    'PublicFloat',
    'PublicFloatValue',
    'ComputedFloat',
    'ComputedMarketFloat',
    'ComputedTreasuryFloat',
]

cash_tags = [
    'Cash','CashAndDueFromBanks','CashAndCashEquivalents',
    'CashAndCashEquivalentsUnrestricted', # this has some funkiness with upper/lower case data
    'CashEquivalentsAtCarryingValue',
    'CashAndCashEquivalentsAtCarryingValue',
    'CashAndCashEquivalentsAtCarryingValueExcludingVariableInterestEntities'
]

debt_tags = [
    'LongTermDebt','LongTermDebtFairValue','LongTermDebtAndCapitalLeaseObligations',
    'DebtAndCapitalLeaseObligations','DebtLongtermAndShorttermCombinedAmount',
    'SecuredDebt','UnsecuredDebt','OperatingLeaseLiabilityNoncurrent',
    'SubordinatedDebt','ConvertibleDebt','LongTermLineOfCredit',
    'OtherBorrowings','NotesAndLoansReceivableNetNoncurrent',
    # NOTE: A MORE ACCURATE ANSWER COMES FROM SUMMING THESE TWO AND COMPARING WITH THE ABOVE (ALREADY-COMBINED) DEBT METRICS
    'LongTermDebtNoncurrent','LongTermDebtCurrent'
]

dera_instant_dict = {
    'float': float_tags,
    'cash': cash_tags,
    'debt': debt_tags,
    'assets': ['Assets'],
}

for what, tags in dera_instant_dict.items():
    tbl = 't_' + what[0]
    
    print(f'Creating instant table `{tbl}` for `{what}`')

    query_usd = generate_instant_dera_usd_query(what, tags)
    # print(query_usd)
    
    query_xyz = generate_instant_dera_xyz_query(what, tags)
    # print(query_xyz)
    
    qres = engine.execute(f"drop table if exists sec_dera.{tbl}")
    # display(qres.fetchall())
    
    qres = engine.execute(query_usd)
    # display(qres.fetchall())
    
    qres = engine.execute(query_xyz)
    # display(qres.fetchall())
    
    query_final = generate_instant_dera_combined_query(tbl, what, [1, 2, 3])
    # print(query_final)
    qres = engine.execute(query_final)
    display(qres.fetchall())

### Fitting Everything Together

The FINANCIALS_BY_LEI table rolls up all the metrics we capture in once place, hopefully with no duplicated rows and only the best facts.

In [ ]:
# Ideally we can replace previously reported numbers with updated numbers.  But not yet.

qres = engine.execute("""
create or replace view sec_dera.financials_by_lei as
select R.name, R.lei, R.cik, if(S.countryinc!='',S.countryinc,S.countryba) as country,
       -- T.tname,
       R.fy, R.ddate, R.sic, revenue_usd, income_usd, float_usd as market_cap_usd, debt_usd, cash_usd, assets_usd
from sec_dera.t_r as R -- left join sec_dera.ticker T on S.cik=T.cik
     join sec_dera.sub as S on R.cik=S.cik and R.fy=S.fy and (S.form='10-K' or S.form='20-F' or S.form='40-F')
     left join sec_dera.t_i as I on R.cik=I.cik and R.ddate=I.ddate
     left join sec_dera.t_f as F on R.cik=F.cik and year(R.ddate)=year(F.ddate)
     left join sec_dera.t_d as D on R.cik=D.cik and R.ddate=D.ddate
     left join sec_dera.t_c as C on R.cik=C.cik and R.ddate=C.ddate
     left join sec_dera.t_a as A on R.cik=A.cik and R.ddate=A.ddate
-- where tname is null or tname not like '%-%'
""")
display(qres.fetchall())

qres = engine.execute("""
select * from sec_dera.financials_by_lei
""")
l = qres.fetchall()
len(l)

In [ ]:
engine.execute("select * from sec_dera.financials_by_lei where lei='8R95QZMKZLJX5Q2XR704' order by ddate desc").fetchall()

### An outstanding GLEIF problem:

How far up the hierarchy should we climb?  For PG&E this really matters...

In [ ]:
l = engine.execute("""
select A.lei, A.parent_name, R.name, date_format(R.fy, '%Y-%m-%d'), date_format(R.ddate, '%Y-%m-%d'), R.revenue_usd
from (select coalesce(G.ultimate_parent_issuer_lei,U.parent_lei) as lei, U.parent_name
      from rmi_20211120.utility_information as U left join gleif_mdt.gleif_direct_issuer_ultimate_issuer as G on U.parent_lei=G.direct_issuer_lei
      group by coalesce(G.ultimate_parent_issuer_lei,U.parent_lei), U.parent_name) as A
     left join sec_dera.t_r as R on A.lei=R.lei
where A.lei is not null
      and (R.lei is null or year(R.fy)=2020)
order by A.parent_name
""").fetchall()

print(len(l))
display(l)

In [ ]:
stop!  Rnadom queries and explorations below...

In [ ]:
display(engine.execute("describe sec_dera.financials_by_lei").fetchall())

display(engine.execute("select * from sec_dera.t_f where lei='2NUNNB7D43COUIRE5295' order by ddate desc").fetchall())

engine.execute("select * from sec_dera.financials_by_lei where lei='2NUNNB7D43COUIRE5295' order by ddate desc").fetchall()

In [ ]:
engine.execute("select count (*) from (select distinct(lei) from sec_dera.sub)").fetchall()

In [ ]:
engine.execute("select S.name, cik, adsh from sec_dera.sub as S where name='DOMINION ENERGY, INC' limit 20").fetchall()

In [ ]:
engine.execute("select * from sec_dera.sub where cik=715957 and form='10-K' limit 20").fetchall()

In [ ]:
engine.execute("select count (*), coreg from sec_dera.num where adsh='0001564590-21-008442' group by coreg limit 20").fetchall()

In [ ]:
engine.execute("select * from sec_dera.num where adsh='0001564590-21-008442' and coreg is null and tag='ProfitLoss' and qtrs=4 order by ddate desc").fetchall()

In [ ]:
engine.execute("select * from sec_dera.sub where adsh='0001564590-21-008442'").fetchall()

In [ ]:
engine.execute("select tag, value from sec_dera.num where adsh='0001564590-21-008442' and coreg='VirginiaElectricAndPowerCompany' and ddate=DATE('2020-12-31') and qtrs=4 order by value desc limit 20").fetchall()

In [ ]:
engine.execute("select coreg, value from sec_dera.num where adsh='0001564590-21-008442' and tag='Revenues' and ddate=DATE('2020-12-31') and qtrs=4 order by value desc limit 20").fetchall()

In [ ]:
engine.execute("select coreg, value from sec_dera.num where adsh='0001564590-21-008444' and tag='RegulatedOperatingRevenue' and ddate=DATE('2020-12-31') order by value desc limit 20").fetchall()

In [ ]:
engine.execute(f"""
select S.adsh, S.cik, S.name, S.lei, T.tname, S.sic, S.fy, N.ddate, N.uom, max(value) as market_cap
from sec_dera.sub as S
     left join sec_dera.float_usd_by_lei X on S.adsh=X.adsh
     join sec_dera.num as N on S.adsh=N.adsh and (X.adsh IS NULL or X.ddate=N.ddate)
     left join sec_dera.ticker T on S.cik=T.cik
     -- we already know we are looking at the correct S.form type
where X.ddate IS NULL
      and value>0
      and N.uom!='USD'
      and coreg is NULL
      -- and qtrs=0
      and (tag='ComputedTreasuryFloat')
group by S.adsh, S.cik, S.name, S.lei, T.tname, S.sic, S.fy, N.ddate, N.uom
""").fetchall()

In [ ]:
engine.execute("select * from sec_dera.float_xyz_by_lei where lei='8R95QZMKZLJX5Q2XR704'").fetchall()

In [ ]:
engine.execute("describe sec_dera.t_f").fetchall()

In [ ]:
engine.execute("select * from sec_dera.float_usd_by_lei where float=54000000.0").fetchall()

In [ ]:
engine.execute("select * from sec_dera.num where adsh like '0001031296-2%' and value=54000000.0").fetchall()